In [2]:
import pandas as pd

DATA_PATH = 'data/'
PERIOD_LABEL = 'validation'

In [3]:
w_weights = [0.5, 0.5]
w_list = list()

for w_file in ['lgb_weights', 'tf_weights']:
    w_list.append(pd.read_csv(DATA_PATH + 'external/' + w_file + '_' + PERIOD_LABEL + '.csv')\
                  .query('id.str.contains("%s")' % PERIOD_LABEL).sort_values('id'))

final_weights = pd.concat([
    w_list[0]['id'].reset_index(drop=True),
    pd.DataFrame((w_list[0].iloc[:,1:]*w_weights[0]).values + (w_list[1].iloc[:,1:]*w_weights[1]).values,
                 columns=['F%s' % c for c in range(1,29)]).reset_index(drop=True)], axis=1)

ids = pd.read_csv('data/raw/sales_train_%s.csv' % PERIOD_LABEL)
ids = ids[['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']]

In [5]:
final_weights = final_weights.merge(ids)

final_weights_agg = pd.melt(
    final_weights.groupby(['store_id', 'dept_id'])[['F%s' % c for c in range(1, 29)]]\
                 .sum().reset_index(),
    id_vars=['store_id', 'dept_id']).rename(columns={'value': 'agg_value'})
final_weights = pd.melt(
    final_weights,
    id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])

final_weights = final_weights.merge(final_weights_agg)
final_weights['value'] = final_weights['value']/final_weights['agg_value']

final_weights = final_weights.pivot_table(index='id', columns=['variable'], values=['value']).reset_index()

final_weights.columns = [c[0] if c[1]=="" else c[1] for c in final_weights.columns.tolist()]
final_weights = final_weights[['id'] + ['F%s' % c for c in range(1, 29)]]
final_weights.to_csv(DATA_PATH + 'external/' + 'ens_weights' + '_' + PERIOD_LABEL + '.csv', index=False)